### Tidy the GEFCom2012 dataset
From 3 file, load_raw.csv, n034_ensemble.csv and temp.csv create the complete.csv file. Use value from ensemble prediction to impute missing value in load_raw, and add temperature values from temp.csv to the final complete dataframe.

#### Read data from csv file

In [1]:
library(tidyr)
library(dplyr)
library(lubridate)

loadRawFile = "GEFCom2012/load_raw.csv"
tempRawFile = "GEFCom2012/temp.csv"
ensemblePredictionFile = "GEFCom2012/n034_ensemble.csv"

#Define a class num.with.commas to probably transform string with comma to number
setClass("num.with.commas")
setAs("character", "num.with.commas", 
        function(from) as.numeric(gsub(",", "", from) ) )

loadRawClasses = c('factor', rep("numeric", 3), rep("num.with.commas", 24))

df = read.csv(loadRawFile, stringsAsFactors=FALSE, colClasses=loadRawClasses)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



#### Tidy loadraw GEFCom2012 data

In [45]:
tidyLoadDf = df %>%  gather(Hour, Consumption, h1:h24) %>%
                 mutate(Hour=as.numeric(substr(Hour, 2, nchar(Hour)))-1) %>%
                 unite(Date, year, month, day, sep='-') %>%
                 unite(DateTime, Date, Hour, sep=' ') %>%
                 mutate(DateTime=as.POSIXct(strptime(DateTime, "%Y-%m-%d %H", tz = "GMT"))) %>%
                 arrange(DateTime, zone_id)


In [46]:
#Spead consumption for each zone into seperate columns, so that we can easily select consumption for each zone later
tidyLoadDf = tidyLoadDf %>% mutate(zone_id=paste0('zone.',as.character(zone_id))) %>%
                      spread(zone_id, Consumption, fill = NA, convert = FALSE)
#Remove 2008/June data, because it's not complete, Create Zone 21 as sum of 20 zones consumption
tidyLoadDf = tidyLoadDf %>% filter(DateTime<as.POSIXct(strptime("2008-06-01 00:00:00", "%Y-%m-%d %H:%M:%S")))
#Add total consumption to zone.21
tidyLoadDf %>% select(zone.1:zone.20) %>% rowSums(na.rm=TRUE) -> tidyLoadDf$zone.21

#### Tidy ensemble prediction GEFCom2012 data

In [47]:
ensembleClasses = c('NULL', 'factor', rep("numeric", 27))
df = read.csv(ensemblePredictionFile, stringsAsFactors=FALSE, colClasses=ensembleClasses)
tidyEnsembleDf = df %>%  gather(Hour, Consumption, h1:h24) %>%
                 mutate(Hour=as.numeric(substr(Hour, 2, nchar(Hour)))-1) %>%
                 unite(Date, year, month, day, sep='-') %>%
                 unite(DateTime, Date, Hour, sep=' ') %>%
                 mutate(DateTime=as.POSIXct(strptime(DateTime, "%Y-%m-%d %H", tz = "GMT"))) %>%
                 arrange(DateTime) %>%
                 mutate(zone_id=paste0('zone.',as.character(zone_id))) %>%
                 spread(zone_id, Consumption, fill = NA, convert = FALSE) %>% #Remove forecast, only need backcast
                 filter(DateTime<as.POSIXct(strptime("2008-06-01 00:00:00", "%Y-%m-%d %H:%M:%S"))) 

In [48]:
head(tidyEnsembleDf)

,DateTime,zone.1,zone.10,zone.11,zone.12,zone.13,zone.14,zone.15,zone.16,zone.17,⋯,zone.2,zone.20,zone.21,zone.3,zone.4,zone.5,zone.6,zone.7,zone.8,zone.9
1,2005-03-06 00:00:00,19756.92438,25033.55209,108900.2974,142251.6476,21516.8568,26596.21596,71789.48446,36732.74398,34992.32603,⋯,162790.9138,86742.34644,1688940.02,175816.4532,520.9084838,8531.423903,170430.3393,174995.3846,3912.874937,74119.88564
2,2005-03-06 01:00:00,19190.47556,24668.02154,106021.6968,136612.7855,20446.99222,25126.27862,69159.73713,36428.806,34019.84176,⋯,158995.8665,84037.83238,1646134.78,171222.157,491.7589507,8347.278861,166499.321,170768.0812,3780.928646,74147.88001
3,2005-03-06 02:00:00,19033.2229,25671.84677,105594.2289,136727.4889,20211.52637,26064.59741,68773.70794,37027.72405,34523.33393,⋯,157209.9721,82865.14351,1646848.246,169635.0138,514.2643139,8559.307027,164707.9693,169445.4386,3711.041964,74223.95562
4,2005-03-06 03:00:00,18264.1712,22956.09595,104902.7827,135124.6118,19725.97278,25779.07654,67874.85368,37549.98063,34748.92179,⋯,160523.186,83666.5785,1652671.216,172211.8922,512.0860038,8589.888315,167755.0607,171982.342,3760.741114,74794.81265
5,2005-03-06 04:00:00,19227.2306,24502.99711,106882.3142,136965.6652,20701.3017,27536.20956,70572.9317,39177.83403,36322.89992,⋯,162036.4273,85877.83067,1691484.125,173720.3804,524.0947217,8836.854655,169769.799,173911.4025,3888.877775,74706.91471
6,2005-03-06 05:00:00,20499.54784,28668.18346,108535.3251,139322.3881,20121.19484,27922.97543,70533.82922,39917.93261,37728.72405,⋯,163818.1623,83506.23914,1715515.615,176265.8241,524.5492641,9408.688655,171226.1775,175929.192,3853.284123,72029.79286


#### Merge ensemble prediction to raw load data, and add temperature data

In [49]:
fullDf = tidyLoadDf
fullDf[is.na(fullDf$zone.1), ] = tidyEnsembleDf[, names(fullDf)]

In [50]:
tempClasses = c('POSIXct', rep("numeric", 11))
tempDf = read.csv(tempRawFile, stringsAsFactors=FALSE, colClasses=tempClasses)
tempDf = tempDf %>% mutate(DateTime = DateTime - minutes(30))
completeDf = left_join(fullDf, tempDf, by="DateTime")
#Check if there is any not complete case
completeDf %>% filter(!complete.cases(.))

In [51]:
write.csv(completeDf, "GEFCom2012/complete.csv", row.names=FALSE)

In [52]:
head(completeDf)

,DateTime,zone.1,zone.10,zone.11,zone.12,zone.13,zone.14,zone.15,zone.16,zone.17,⋯,T02,T03,T04,T05,T06,T07,T08,T09,T10,T11
1,2004-01-01 00:00:00,16853,23339,90700,118378,20673,21791,65970,28752,30645,⋯,38,44,45,42,44,45,43,41,42,36
2,2004-01-01 01:00:00,16450,22100,86699,112480,19666,21400,64600,27851,30461,⋯,36,42,43,42,43,44,44,39,43,32
3,2004-01-01 02:00:00,16517,21376,84243,108435,19020,20998,63843,27631,30197,⋯,35,40,41,40,42,41,42,36,43,31
4,2004-01-01 03:00:00,16873,21335,84285,107224,18841,21214,64023,27986,30264,⋯,30,36,37,39,38,40,34,35,39,30
5,2004-01-01 04:00:00,17064,21564,86087,108870,19310,21830,65679,29160,30907,⋯,30,34,33,40,38,35,30,33,35,34
6,2004-01-01 05:00:00,17727,22241,90210,112395,19415,21794,63305,29226,31617,⋯,29,32,32,41,37,35,35,36,35,35
